In [1]:
# SELECT DISTINCT ?manVal ?wifeVal ?womanVal ?husbandVal WHERE {
#     {
#         ?man a tbio:Person .
#         ?man tbio:hasWife ?wife .
#     }
#     UNION
#     {
#         ?woman a tbio:Person .
#         ?woman tbio:hasHusband ?husband .
#     }
#     BIND(STR(?man) AS ?manStr) .
#     BIND(REPLACE(?manStr, "http://tbio.orient.cas.cz#", "") AS ?manVal) .
#     BIND(STR(?wife) AS ?wifeStr) .
#     BIND(REPLACE(?wifeStr, "http://tbio.orient.cas.cz#", "") AS ?wifeVal) .
#     BIND(STR(?woman) AS ?womanStr) .
#     BIND(REPLACE(?womanStr, "http://tbio.orient.cas.cz#", "") AS ?womanVal) .
#     BIND(STR(?husband) AS ?husbandStr) .
#     BIND(REPLACE(?husbandStr, "http://tbio.orient.cas.cz#", "") AS ?husbandVal) .
# }

In [2]:
import pandas as pd

# Get marriages

In [3]:
filepath = '/Volumes/backup_128G/z_repository/TBIO_data/RequestsFromTana/20190515'

filename = 'marriages.tsv'

read_filename = '{0}/{1}'.format(filepath, filename)

In [4]:
marriageDf = pd.read_csv(read_filename, delimiter='\t')
marriageDf.fillna('', inplace=True)
marriageDf.shape, marriageDf.head()

((3353, 4),   ?manVal ?wifeVal ?womanVal ?husbandVal
 0     郭火炎       施珍                      
 1     羅享錦      吳綢妹                      
 2     林熊祥      陳師桓                      
 3     林熊祥      蕭翠香                      
 4     丁瑞圖       魏罔                      )

# To unique spouses

In [5]:
startId = 10000
personIds = {}
marriageDic = {}
for idx in range(0, len(marriageDf)):
    row = marriageDf.loc[idx]

    man = str(row['?manVal'])
    wife = str(row['?wifeVal'])
    woman = str(row['?womanVal'])
    husband = str(row['?husbandVal'])
    
    if man != '' and man not in personIds:
        personIds[man] = startId
        startId += 1
    if wife != '' and wife not in personIds:
        personIds[wife] = startId
        startId += 1
    if woman != '' and woman not in personIds:
        personIds[woman] = startId
        startId += 1
    if husband != '' and husband not in personIds:
        personIds[husband] = startId
        startId += 1
        
    if man not in marriageDic:
        marriageDic[man] = [wife]
    elif wife not in marriageDic[man]:
        marriageDic[man].append(wife)
#     else:
#         print("man WRONG:", man, wife)

    if wife not in marriageDic:
        marriageDic[wife] = [man]
    elif man not in marriageDic[wife]:
        marriageDic[wife].append(man)
#     else:
#         print("wife WRONG:", wife, man)
        
    if woman not in marriageDic:
        marriageDic[woman] = [husband]
    elif husband not in marriageDic[woman]:
        marriageDic[woman].append(husband)
#     else:
#         print("woman WRONG:", woman, husband)
        
    if husband not in marriageDic:
        marriageDic[husband] = [woman]
    elif woman not in marriageDic[husband]:
        marriageDic[husband].append(woman)
#     else:
#         print("husband WRONG:", husband, woman)
        
# marriageDic

In [6]:
len(personIds)

3922

In [7]:
personDf = pd.DataFrame(personIds, index=['ID']).T
personDf.head()

,ID
郭火炎,10000
施珍,10001
羅享錦,10002
吳綢妹,10003
林熊祥,10004


In [8]:
write_nodes_to = '{0}/{1}'.format(filepath, 'nodes_person_20190516_v2.xlsx')
personDf.to_excel(write_nodes_to)

# Read person-family map table

In [9]:
familymembers = 'Familymembers.xlsx'

read_familymembers = '{0}/{1}'.format(filepath, familymembers)

In [10]:
fmDf = pd.read_excel(read_familymembers)
fmDf.shape, fmDf.head()

((4546, 2),   personStr familyStr
 0       何佩娟      何傳家系
 1       何佩珊      何傳家系
 2       何佩珍      何傳家系
 3        何傳      何傳家系
 4       何培庭      何傳家系)

In [11]:
startId = 20000
familyIds = {}
fmDic = {}
for idx in range(0, len(fmDf)):
    row = fmDf.loc[idx]

    person = str(row['personStr'])
    family = str(row['familyStr'])
    
    if family not in familyIds:
        familyIds[family] = startId
        startId += 1
        
    if person not in fmDic:
        fmDic[person] = family
    elif family != fmDic[person]:
        print("Dup:", person, family, fmDic[person])

# fmDic

In [12]:
len(familyIds)

1222

In [13]:
familyDf = pd.DataFrame(familyIds, index=['ID']).T
familyDf.head()

,ID
何傳家系,20000
李招治家族,20001
李火木家族,20002
林妙真家族,20003
林富香家族,20004


In [14]:
write_nodes_to = '{0}/{1}'.format(filepath, 'nodes_family_20190516_v2.xlsx')
familyDf.to_excel(write_nodes_to)

# results

### `Source (Family)` | `Target(Family)`| `Type(Undirected)` | `Person/Source` | `Person/target`

In [15]:
def getFamilyName(INperson):
    if INperson not in fmDic:
#         print(INperson, " Not found!")
        return ''
    return fmDic[INperson]

def getPersonId(INperson):
    if INperson not in personIds:
        return 0
    return personIds[INperson]

def getFamilyId(INfamily):
    if INfamily not in familyIds:
        return 0
    return familyIds[INfamily]

resList = []
for sPerson in marriageDic:
    spouses = marriageDic[sPerson]
    for tPerson in spouses:
        sFamily = getFamilyName(sPerson)
        tFamily = getFamilyName(tPerson)
        
        if sFamily == '' or tFamily == '':
#             print(fPerson, fFamily, sPerson, sFamily)
            continue

        sPersonId = getPersonId(sPerson)
        tPersonId = getPersonId(tPerson)
        sFamilyId = getFamilyId(sFamily)
        tFamilyId = getFamilyId(tFamily)
        resList.append([sFamilyId, tFamilyId, 'Undirected', sPersonId, tPersonId])

print(len(resList))

2573


In [16]:
resDf = pd.DataFrame(resList, columns=['SourceFamily', 'TargetFamily', 'Type', 
                                       'SourcePerson', 'TargetPerson'])
resDf.drop_duplicates(keep='first', inplace=True)
resDf.sort_values(by=['SourceFamily', 'TargetFamily', 'SourcePerson', 'TargetPerson'], inplace=True)
resDf.head()

,SourceFamily,TargetFamily,Type,SourcePerson,TargetPerson
91,20000,20001,Undirected,10300,10301
1638,20000,20002,Undirected,12970,12969
1642,20000,20003,Undirected,12974,12975
92,20000,20004,Undirected,10300,10302
1634,20000,20005,Undirected,12966,12965


In [17]:
print(len(resDf))

2573


In [18]:
write_file_to = '{0}/{1}'.format(filepath, 'marriages_20190516_v2.xlsx')
resDf.to_excel(write_file_to, index=False)